In [1]:
import tensorflow as tf
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
assert sys.version_info.major == 3
np.random.seed(0)

# 2 - TensorFlow
### 2.1 - Overview

Now we're going to introduce the TensorFlow (TF) framework. This is very much using a sledgehammer to crack a nut, but introducing it in this simple example provides an nice opportunity to explain it. This introduction is largely taken from the [official](https://www.tensorflow.org/get_started/get_started) getting started guide.

##### 2.1.1 - Tensors
TF takes its name from the **tensor** which is a data structure of primative values, which are shaped into an array:

```Python
3 # a rank 0 tensor; this is a scalar with shape []
[1., 2., 3.] # a rank 1 tensor; this is a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]
```

Think. What would a tensor of shape [1, 2, 3, 4] look like? 

Hint: the dimensions are the same as in numpy, so you can play around with that to get a feel for higher-dimensional matrices.

In [2]:
a = np.array([[1., 2., 3.], [4., 5., 6.]])
b = np.array([[[1., 2., 3.]], [[7., 8., 9.]]])
print(a.shape)
print(b.shape)

(2, 3)
(2, 1, 3)


It's hard to visualise a 4D-tensor, however they are useful in real-world problems such as image classification.

The computer 'sees' an image as a matrix of values, where each value represents one pixel. So imagine a 2D image of 20px $\times$ 20px, this is Tensor of shape [20, 20]. Now imagine we introduce Red, Green, and Blue colour channels. We can represent each channel as a separate matrix of 20px $\times$ 20px, and we can stack these onto each other to produce a Tensor of shape [20, 20, 3].

Now, imagine we have a batch of 50 images. Well, we can stack these again to form a Tensor of shape [50, 32, 32, 3]. Now suppose there are 10 batches of 50 images. Could we stack these? Sure, and we would have a Tensor of shape [10, 50, 32, 32, 3].

Hopefully this helps in visualising past 3 dimensions.

#### 2.1.2 - TF structure
TF is a framework which is designed to facilitate the development of deep learning, and can be thought of as creating a a program in two parts:

1. Building the computational graph
2. Running the computational graph

A computational graph can be though of as a series of **nodes** which take in 0 or more tensors and output a tensor. This could be a node which creates a constant tensor (takes in 0 tensors and outputs 1 tensor), or a node which multiplies tensors together (takes in $n$ tensors, and outputs 1 tensor). In the latter example, these are known as **operations**.

Think of the computational graph as being like a series of pipes, that split and rejoin at various points. Running the computational graph would be like pouring water into the pipes, and seeing it flow around the pipework. Although instead of water, we have data.

Let's start with some simple examples.

### 2.2 - Getting started

Let's build a node which creates a constant:

In [4]:
node_1 = tf.constant(3.0, dtype=tf.float32, name='node_1')

That seems like a lot of code to produce a constant floating point number, and it is, especially if you are used to Python. However it helps to construct the graph, since it is very explicit. From the docs:
```
tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)
```

So we can see that there is a slightly more compact way to create the node, if we wanted:

```
node_1 = tf.constant(3.0)
```

However, at this stage it helps to get into the habit of being explicit, _especially_ with naming.

What happens when we print this node? Should we see 3.0?

In [5]:
print(node_1)

Tensor("node_1_1:0", shape=(), dtype=float32)


Hmm, that's odd. We've got a tensor, with the name of the node, but not much else. This is because we have only completed the first part of the program: building the computational graph. We still need to run the graph.

To do this, we need to first create a session. This is an object which encapsulates the environment in which operations are exececuted:

In [6]:
sess = tf.Session()

We can then use this object's ```run``` method to execute our node.

In [7]:
sess.run(node_1)

3.0

We now need to close our session, to avoid using resources that may be required by other sessions.

In [8]:
sess.close()

Because of this, it is more common to construct sessions using the ```with``` statement:

In [9]:
with tf.Session() as sess:
    print(sess.run(node_1))

3.0


This automatically closes the session.

### 2.2 - Operations

Now we are going to build a very simple graph, which adds two numbers together. 

First create our constants:

In [10]:
node_1 = tf.constant(3.0, dtype=tf.float32, name='node_1')
node_2 = tf.constant(5.0, dtype=tf.float32, name='node_2')

Now, we are going to create a node to add these two nodes together:

In [11]:
node_3 = tf.add(node_1, node_2)

Remember, we have just built the graph at the moment. Now, let's run it:

In [12]:
with tf.Session() as sess:
    print(sess.run(node_3))

8.0


Our first (proper) graph works! At this stage it may not be clear exactly why we need all this code to add two numbers together, but hopefully it will become clearer shortly.

In [20]:
# OPTIONAL
# Multiply the two numbers together

### YOUR CODE HERE ###

### 2.3 - Placeholders

So far we've seen how to create constants, but you may be thinking wondering about the analogy from above regarding pouring water into the pipes. To do this 'pouring' we need to define **placeholders**.

These are nodes which accept external values, which are fed (or poured) when we run the graph.

In [13]:
node_1 = tf.placeholder(dtype=tf.float32, name='node_1')
node_2 = tf.placeholder(dtype=tf.float32, name='node_2')

As you can see, they follow a similar syntax as the constants, however they do not have a value associated with them yet.

To give these nodes values, we need to supply something called a **feed dictionary**, which is a Python dictionary mapping the nodes to the input values.

We'll present the example from above, where we add the two nodes together.

In [14]:
node_3 = tf.add(node_1, node_2)

with tf.Session() as sess:
    print(sess.run(node_3, feed_dict={node_1: 3.0, node_2: 5.0}))

8.0


Great! We've now built a computational graph, supplied data to it, and ran it. We're closing in on having a solid understanding of TensorFlow now. But before we move on to using it for linear regression, there's one more important node to be aware of.

### 2.4 - Variables